In [6]:
# Tools creation

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper


In [7]:
# Used the inbuilt tools of Wikipedia

wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)
wiki_tool.name

'wikipedia'

In [8]:
# Used inbuilt tool: Arxiv

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [9]:
tools=[wiki_tool, arxiv_tool]


In [10]:

## Create a Custom tool for RAG Tool 
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
loader = WebBaseLoader("https://docs.langsmith.com/")
docs=loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
documents=splitter.split_documents(docs)[0:200]
vector_db=FAISS.from_documents(documents, OllamaEmbeddings())
retriver = vector_db.as_retriever()
retriver

In [12]:
# Convert retriever to tool

from langchain.tools.retriever import create_retriever_tool
retriver_tool=create_retriever_tool(retriever, 'langsmith-tool', 'Search info about langsmit')
retriver_tool.name

In [13]:
# Run all these tools with agents and LLM Models
from dotenv import load_dotenv
load_dotenv()
import os
from langchain_groq import ChatGroq
groq_api_key=os.getenv('GROQ_API_KEY')

llm = ChatGroq(groq_api_key=groq_api_key,model='Llama3-8b-8192')


In [14]:
## Ready made prompt template from langchain

from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\HP\miniconda3\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
## Agents

from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm, tools, prompt)
agent


RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [16]:
# Agent executer

from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [17]:
agent_executor.invoke({"input": "Tell me about machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gen
Invoking: `wikipedia` with `{'query': 'Machine learning#Types'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gen
Invoking: `arxiv` with `{'query': 'machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton Univer
Invoking: `wikipedia` with `{'query': 'Optimization for Machine Learning'}`


Page: Hyperparameter optimization
Summary: In machine learning, hyperparameter optimization o

{'input': 'Tell me about machine learning',
 'output': 'Tool use failed: no valid JSON object found in message content'}

In [18]:
agent_executor.invoke({"input": "Tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Langsmith'}`


No good Wikipedia Search Result was found
Invoking: `arxiv` with `{'query': 'Langsmith'}`
responded: It seems like the Wikipedia search didn't yield a good result. Let me try again!



Published: 2020-10-09
Title: Langsmith: An Interactive Academic Text Revision System
Authors: Takumi Ito, Tatsuki Kuribayashi, Masatoshi Hidaka, Jun Suzuki, Kentaro Inui
Summary: Despite the current dHere's a response:

Langsmith is an interactive academic text revision system that helps authors improve the quality of their written work. It was developed by Takumi Ito, Tatsuki Kuribayashi, Masatoshi Hidaka, Jun Suzuki, and Kentaro Inui in 2020. The system provides a interactive framework for authors to revise their texts, offering suggestions for improvement and allowing them to track changes made during the revision process.

> Finished chain.


{'input': 'Tell me about langsmith',
 'output': "Here's a response:\n\nLangsmith is an interactive academic text revision system that helps authors improve the quality of their written work. It was developed by Takumi Ito, Tatsuki Kuribayashi, Masatoshi Hidaka, Jun Suzuki, and Kentaro Inui in 2020. The system provides a interactive framework for authors to revise their texts, offering suggestions for improvement and allowing them to track changes made during the revision process."}